In [ ]:
%%capture
!pip install fastapi nest_asyncio pyngrok uvicorn monai python-multipart

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/Machine-Learning-Biomedicine/PankVision-3D/model/get_model.py /content
!cp /content/drive/MyDrive/Machine-Learning-Biomedicine/PankVision-3D/preprocess/final_preprocess.py /content

In [14]:
from fastapi import FastAPI, UploadFile
from pydantic import BaseModel
import nest_asyncio
from pyngrok import ngrok
import uvicorn

import numpy as np
from PIL import Image
from io import BytesIO
import nibabel as nib

In [ ]:
from monai.utils import first, set_determinism
from monai.transforms import(
    Compose,
    AddChanneld,
    LoadImaged,
    Resized,
    ToTensord,
    Spacingd,
    Orientationd,
    ScaleIntensityRanged,
    CropForegroundd,
    Activations,
)

from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.data import CacheDataset, DataLoader, Dataset

import torch
import matplotlib.pyplot as plt

import os
from glob import glob
import numpy as np

from monai.inferers import sliding_window_inference
from final_preprocess import prepare
from get_model import get_model

In [ ]:
model_dir = '/content/drive/MyDrive/Machine-Learning-Biomedicine/PankVision-3D/results/dataset-007/v6dynunet'

In [15]:
def convert_to_image(file_contents):
    """
    Convert the uploaded file to an image format.
    This function assumes that the uploaded file is an image file.
    """
    # Convert the file contents to a BytesIO object
    byte_stream = BytesIO(file_contents)

    # Open the byte stream as an image
    image = Image.open(byte_stream)

    # Convert the image to a NumPy array and return it
    return np.array(image)

def preprocess_nifti_image(image_path):
    # Define the transformations
    combined_transforms = Compose(
        [
            LoadImaged(keys=["vol"]),
            AddChanneld(keys=["vol"]),
            Spacingd(keys=["vol"], pixdim=(1.5, 1.5, 1.0), mode=("bilinear")),
            Orientationd(keys=["vol"], axcodes="RAS"),
            ScaleIntensityRanged(keys=["vol"], a_min=-200, a_max=200,b_min=0.0, b_max=1.0, clip=True),
            CropForegroundd(keys=['vol'], source_key='vol'),
            Resized(keys=["vol"], spatial_size=[128, 128, 64]),
            ToTensord(keys=["vol"]),
        ]
    )

    # Create a dataset with the image file and apply the transformations
    data = [{"vol": image_path}]
    ds = Dataset(data=data, transform=combined_transforms)

    # Create a DataLoader
    loader = DataLoader(ds, batch_size=1)

    return loader

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
args = {
    'model_name': 'DynUNet',
    'pretrained': True,
    'dropout': 0.1
}
model = get_model(args)
model = model.to(device)
model.load_state_dict(torch.load(os.path.join(model_dir, "best_metric_model.pth")))
model.eval()

def run_model(image):
    """
    Run the model on the preprocessed image and get the results.
    """
    # Ensure the model is in evaluation mode
    model.eval()

    # Move the image to the device used by the model
    image = image.to(device)

    # Add an extra dimension for the batch size
    image = image.unsqueeze(0)

    # Run the model on the image
    with torch.no_grad():
        output = model(image)

    # Convert the output to a NumPy array and return it
    return output.cpu().numpy()

app = FastAPI()

class InputData(BaseModel):
    file: UploadFile

# Define the FastAPI endpoint for uploading files
@app.post("/upload")
async def upload_file(file: UploadFile):
    # Read the file
    contents = await file.read()

    # Convert the file contents to an image (this will depend on your file format)
    image = convert_to_image(contents)

    # Preprocess the image
    preprocessed_image = preprocess_nifti_image(image)

    # Run the model on the preprocessed image and get the results
    results = run_model(preprocessed_image)

    # Return the results
    return {"results": results}

nest_asyncio.apply()

url = ngrok.connect(80)
print('Public URL:', url)
uvicorn.run(app, host='0.0.0.0', port=80)
